<a href="https://colab.research.google.com/github/biku1998/NLP-Notebooks/blob/master/Inshorts-Topic-Modeling/Inshorts_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inshort News Topic Modeling Using Gensim
---
### Notebook Outlines
* **Scrape the news data from Inshorts Using `beautifulSoup` and `selenium`**
* **Process the collected data**
* **Build a topic model**


**If you are new to topic modeling check out this** <a href = "https://colab.research.google.com/github/biku1998/NLP-Notebooks/blob/master/NLP_04_Topic_Modeling_LDA_Gensim.ipynb"><button>**Notebook**</button>

- Data Collection
    
    For data collection we will use bs4 and selenium as if you look on the inshort website [https://inshorts.com/en/read]. We have to click on the button at the bottom of the page to fetch more news.

In [0]:
path_to_selenium_driver = "C:/chromedriver_win32/chromedriver.exe"

In [0]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import time

In [0]:
def news_collector(ntimes,url,path_to_selenium_driver):
    """
    to collect data from inshorts
    parameters :
    ntime : how many times `load more` button will be clicked ?
    url : url of the website
    path_to_selenium_driver : path to selenium driver
    """
    driver = webdriver.Chrome(path_to_selenium_driver)
    driver.get(url)
    
    
    
    # find the button
    load_more_button  = driver.find_element_by_xpath("//*[@id='load-more-btn']")
    
    # click on the button n times
    print("Performing the click event !!")
    
    for i in range(ntimes):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1) # a great hack
        load_more_button.click()
    
    # get the news blocks
    news_page_source = driver.page_source
    
    driver.close()
    
    return news_page_source
    

In [0]:
def get_news_from_page_source(page_source):
    """
    to return a clean formated data from a page source provided by selenium
    """
    
    soup = BeautifulSoup(page_source)
    
    news_blocks = soup.find_all("div","news-card z-depth-1")
    
    print(f"Total {len(news_blocks)} news blocks found !")
    
#     print(news_blocks[0].prettify()) # for debug

    # create structure for dataFrame
    
    df_columns = ['news_title',"news_text"]
    
    rows = []
    
    print("Extracting news from html !")
    for block in news_blocks:
        
        news_title = block.find(itemprop="headline").text
        
        news_text = block.find("div","news-card-content news-right-box").text
        
        # append the data in rows
        
        rows.append((news_title,news_text))
        
        # for debug only
        
#         print("-----------------------------------")
        
#         print(f"news title : ",news_title)
#         print(f"news text  :", news_text)
        
#         print("-----------------------------------")

    return pd.DataFrame(data = rows,columns  = df_columns)    

In [0]:
def dataloader(ntimes,url,path_to_selenium_driver):
    """
    will use the above functions to fetch and collect data and return a DataFrame
    """
    
    news_blocks_page_source = news_collector(ntimes,url,path_to_selenium_driver)
    
    df = get_news_from_page_source(news_blocks_page_source)
    
    return df
    

In [0]:
df_news = dataloader(20,"https://inshorts.com/en/read",path_to_selenium_driver)

In [0]:
df_news.shape

In [0]:
df_news.head()

In [0]:
df_news.to_csv("./inshorts_news_data.csv",index = False)

In [0]:
# load the data

df_news = pd.read_csv("./inshorts_news_data.csv")

In [0]:
df_news.head()

,news_title,news_text
0,Passenger train services to restart from May 1...,\nThe Indian Railways will gradually restart p...
1,Former PM Manmohan Singh admitted to AIIMS aft...,\nFormer Prime Minister Manmohan Singh has bee...
2,"Coronavirus cases rise to 3,814 in Rajasthan, ...",\nRajasthan on Sunday reported 106 new coronav...
3,India develops its 1st indigenous antibody det...,\nThe National Institute of Virology in Pune h...
4,What are the guidelines for passengers ahead o...,\nBooking for the special passenger trains tha...


In [0]:
print(df_news['news_title'].values[0])

Passenger train services to restart from May 12 with 15 pairs of trains: Govt


In [0]:
print(df_news['news_text'].values[0])


The Indian Railways will gradually restart passenger train operations from May 12, initially with 15 pairs of trains (30 return journeys), it announced. These trains will be run as special trains from New Delhi connecting 15 important cities including Bengaluru, Chennai, Mumbai and Ahmedabad, Railways tweeted. Booking for reservation in these trains will start at 4 pm on May 11.

short by Anmol Sharma / 
      09:00 pm on 10 May




**Note**
* we have to remove "short by Anmol Sharma / 09:00 pm on 10 May" phrase from the end of the news text, as it contains no information

In [0]:
def remove_text(news_text):
    return news_text.strip().split("short")[0].strip()

In [0]:
df_news['news_text'] = df_news['news_text'].apply(remove_text)

In [0]:
print(df_news['news_text'].values[0])

The Indian Railways will gradually restart passenger train operations from May 12, initially with 15 pairs of trains (30 return journeys), it announced. These trains will be run as special trains from New Delhi connecting 15 important cities including Bengaluru, Chennai, Mumbai and Ahmedabad, Railways tweeted. Booking for reservation in these trains will start at 4 pm on May 11.


In [0]:
# pre-process imports

import numpy as np
import matplotlib.pyplot as plt
import os,re
from gensim.parsing.preprocessing import remove_stopwords,strip_punctuation
import nltk
nltk.download("wordnet")

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sourabh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
def pre_process_news(texts):
    
    # tokenization
    
    texts = [re.findall(r'\w+', line.lower()) for line in texts]
    
    # remove stopwords
    
    texts = [remove_stopwords(' '.join(line)).split() for line in texts]
    
    # remove punctuation
    
    texts = [strip_punctuation(' '.join(line)).split() for line in texts]
    
    # remove words that are only 1-2 character
    texts = [[token for token in line if len(token) > 2] for line in texts]
    
    # remove numbers
    texts = [[token for token in line if not token.isnumeric()] for line in texts]
    
    # lemmatization 
 
    lemmatizer = WordNetLemmatizer()
    texts = [[word for word in lemmatizer.lemmatize(' '.join(line), pos='v').split()] for line in texts]
    
    return texts

In [0]:
news_cleaned = pre_process_news(df_news['news_text'].values)

In [0]:
len(news_cleaned)

532

In [0]:
news_cleaned[0]

['indian',
 'railways',
 'gradually',
 'restart',
 'passenger',
 'train',
 'operations',
 'initially',
 'pairs',
 'trains',
 'return',
 'journeys',
 'announced',
 'trains',
 'run',
 'special',
 'trains',
 'new',
 'delhi',
 'connecting',
 'important',
 'cities',
 'including',
 'bengaluru',
 'chennai',
 'mumbai',
 'ahmedabad',
 'railways',
 'tweeted',
 'booking',
 'reservation',
 'trains',
 'start']

In [0]:
# bi gram collection detection

from gensim.models.phrases import Phraser,Phrases

# train the bigram det

phrases = Phrases(news_cleaned,min_count = 1,threshold = 0.8,scoring = 'npmi')

In [0]:
# now we create a transformer that will use the above trained model to transform new words

bi_gram = Phraser(phrases)

In [0]:
# merging detected collocations with data

news_cleaned = list(bi_gram[news_cleaned])

In [0]:
# creating a numerical mapping for each word

from gensim.corpora import Dictionary

In [0]:
dictionary = Dictionary(news_cleaned)

In [0]:
# remove rare words and most common words to improve our topic modeling

# Filter out words that occur less than 10 documents, or more than 60% of the documents.

dictionary.filter_extremes(no_below=10, no_above=0.6)

In [0]:
#  transform the documents to a vectorized form. We simply compute the frequency of each word, including the bigrams.

corpus = [dictionary.doc2bow(text) for text in news_cleaned]

In [0]:
doc_number = 0
corpus[doc_number] # A document is represented as a list of tuples of (vocab ID, frequency) for each word.

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 4),
 (10, 1)]

In [0]:
# train the lda model

from gensim.models import LdaModel


In [0]:
lda_model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 10, \
                      passes = 50, chunksize = 1500,iterations = 200,alpha = "auto")

In [0]:
# once the model is trained we save the model

if os.path.isdir("./topic_models") == False:
    os.mkdir("./topic_models")
    
lda_model.save("./topic_models/lda_model")

In [0]:
# To load the modal 

# lda_model  = lda_model.load("./topic_models/lda_model")

In [0]:
lda_model.show_topics(num_topics = 5)

[(5,
  '0.100*"coronavirus" + 0.082*"said" + 0.055*"country" + 0.048*"china" + 0.041*"cases" + 0.041*"reported" + 0.037*"covid" + 0.033*"health" + 0.029*"ministry" + 0.027*"city"'),
 (0,
  '0.068*"world" + 0.055*"women" + 0.054*"million" + 0.044*"coronavirus" + 0.033*"group" + 0.033*"video" + 0.032*"app" + 0.032*"trump" + 0.032*"report" + 0.030*"april"'),
 (4,
  '0.110*"cases" + 0.084*"coronavirus" + 0.056*"state" + 0.043*"reported" + 0.036*"patients" + 0.034*"total_number" + 0.033*"health" + 0.029*"new" + 0.028*"taking" + 0.024*"covid"'),
 (7,
  '0.105*"said" + 0.062*"coronavirus" + 0.050*"covid" + 0.049*"pandemic" + 0.045*"added" + 0.041*"amid" + 0.032*"people" + 0.031*"lockdown" + 0.030*"government" + 0.030*"friday"'),
 (9,
  '0.150*"said" + 0.103*"india" + 0.094*"added" + 0.032*"indian" + 0.030*"cricket" + 0.028*"team" + 0.028*"australia" + 0.025*"stated" + 0.024*"played" + 0.021*"years"')]

In [0]:
import pyLDAvis.gensim

# set the notebook model
pyLDAvis.enable_notebook()

In [0]:
import warnings
warnings.filterwarnings("ignore")

**To understand what's going on in this plot refer this** <a href = "https://colab.research.google.com/github/biku1998/NLP-Notebooks/blob/master/NLP_04_Topic_Modeling_LDA_Gensim.ipynb"><button>**Notebook**</button>

In [0]:
pyLDAvis.gensim.prepare(lda_model,corpus,dictionary,sort_topics = False)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.090177  0.029896       1        1   5.927916
1      0.070521 -0.058378       2        1   7.144493
2      0.002207 -0.220900       3        1   9.117367
3      0.239453  0.097688       4        1   7.017550
4     -0.250985  0.097723       5        1  23.171907
5     -0.199470  0.091312       6        1   8.493396
6     -0.035953 -0.209855       7        1   6.733599
7     -0.017574  0.006373       8        1  12.983152
8      0.106891  0.144772       9        1   8.200565
9      0.175088  0.021371      10        1  11.210058, topic_info=    Category        Freq          Term       Total  loglift  logprob
16   Default  168.000000         cases  168.000000  30.0000  30.0000
35   Default   80.000000         india   80.000000  29.0000  29.0000
37   Default  426.000000          said  426.000000  28.0000  28.0000
144  Default   54.000000         crore   54.000000  27.0000  27.0000
82   Default   56.000000        police   56.000000  26.0000  26.0000
17   Default  258.000000   coronavirus  258.000000  25.0000  25.0000
26   Default  119.000000         state  119.000000  24.0000  24.0000
68   Default  156.000000         added  156.000000  23.0000  23.0000
53   Default   30.000000        mother   30.000000  22.0000  22.0000
25   Default   88.000000      reported   88.000000  21.0000  21.0000
146  Default   35.000000          year   35.000000  20.0000  20.0000
75   Default   26.000000         world   26.000000  19.0000  19.0000
78   Default   35.000000       country   35.000000  18.0000  18.0000
70   Default   64.000000      lockdown   64.000000  17.0000  17.0000
32   Default   66.000000        health   66.000000  16.0000  16.0000
130  Default   84.000000    government   84.000000  15.0000  15.0000
51   Default   30.000000           day   30.000000  14.0000  14.0000
69   Default  101.000000         covid  101.000000  13.0000  13.0000
86   Default   47.000000      pandemic   47.000000  12.0000  12.0000
187  Default   31.000000          film   31.000000  11.0000  11.0000
84   Default   44.000000          amid   44.000000  10.0000  10.0000
117  Default   34.000000          like   34.000000   9.0000   9.0000
30   Default   45.000000  total_number   45.000000   8.0000   8.0000
22   Default   55.000000      patients   55.000000   7.0000   7.0000
63   Default   25.000000         china   25.000000   6.0000   6.0000
109  Default   24.000000       million   24.000000   5.0000   5.0000
13   Default   36.000000         singh   36.000000   4.0000   4.0000
169  Default   43.000000          home   43.000000   3.0000   3.0000
105  Default   19.000000         women   19.000000   2.0000   2.0000
92   Default   20.000000     employees   20.000000   1.0000   1.0000
..       ...         ...           ...         ...      ...      ...
155  Topic10   11.518504          play   14.455737   1.9612  -4.0345
96   Topic10   18.425150          team   23.123720   1.9612  -3.5647
145  Topic10   10.689451      revealed   13.529013   1.9528  -4.1092
131  Topic10   13.623517         years   18.323914   1.8919  -3.8667
186  Topic10   11.763658          khan   16.371803   1.8578  -4.0134
164  Topic10    8.774770          know   12.602639   1.8263  -4.3066
107  Topic10    9.113896          data   13.483250   1.7967  -4.2687
3    Topic10   20.971399        indian   31.693798   1.7754  -3.4353
156  Topic10    8.725136           way   13.462061   1.7547  -4.3123
104  Topic10    7.260442         think   11.568070   1.7226  -4.4960
158  Topic10    7.860979       playing   13.445640   1.6516  -4.4165
97   Topic10    7.527094          test   13.440104   1.6086  -4.4600
140  Topic10   11.764221         going   23.082062   1.5144  -4.0134
95   Topic10   16.298470        stated   32.974621   1.4837  -3.6874
154  Topic10    5.474607          good   13.458685   1.2889  -4.7783
143  Topic10    8.344554       company   21.056446   1.2628  -4.35

In [0]:
### Document Clustering using LDA on Tensorboard

# Get document topics
all_topics = lda_model.get_document_topics(corpus, minimum_probability=0)
all_topics[0]

[(0, 0.0028396605),
 (1, 0.0030884051),
 (2, 0.6604015),
 (3, 0.0031240103),
 (4, 0.004412665),
 (5, 0.002818684),
 (6, 0.003101953),
 (7, 0.0045325384),
 (8, 0.31105405),
 (9, 0.0046265377)]

In [0]:
# create file for tensors(vectors)

with open('doc_lda_tensor.tsv','w') as w:
    for doc_topics in all_topics:
        for topics in doc_topics:
            w.write(str(topics[1])+ "\t") # store the probablity value
        w.write("\n")

In [0]:
# create file for metadata(documet titles)
with open('doc_lda_metadata.tsv','w',encoding="utf-8") as w:
    for doc_id in range(len(all_topics)):
        w.write(df_news.news_title[doc_id] + "\n")

* Now open - http://projector.tensorflow.org/
* upload both the files
* Upload both the files using the `load` button on the left

**Below is the quick view of the news cluster that we have made**

---

<img src = "./news_cluster.gif">